In [5]:
from music21 import converter, instrument, note, chord, stream
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [6]:


# Función para cargar y procesar archivos MIDI en secuencias de notas
def load_midi_files(data_path):
    notes = []
    for artist_folder in os.listdir(data_path):
        artist_path = os.path.join(data_path, artist_folder)
        for midi_file in os.listdir(artist_path):
            midi_path = os.path.join(artist_path, midi_file)
            try:
                midi_data = converter.parse(midi_path)
                parts = instrument.partitionByInstrument(midi_data)
                notes_in_song = []
                for part in parts.parts:
                    if 'Piano' in str(part):  # Filtrar partes de piano
                        for element in part.flat.notes:
                            if isinstance(element, note.Note):  # Notas individuales
                                notes_in_song.append(str(element.pitch))
                            elif isinstance(element, chord.Chord):  # Acordes
                                notes_in_song.append('.'.join(str(n.pitch) for n in element.notes))
                if notes_in_song:
                    notes.append(notes_in_song)
            except Exception as e:
                print(f"Error al cargar {midi_path}: {e}")
                continue
    return notes

# Cargar datos MIDI
#data_path = "midi"
#notes = load_midi_files(data_path)



In [7]:
def prepare_sequences(notes, sequence_length):
    unique_notes = sorted(set(item for song in notes for item in song))
    note_to_int = {note: num for num, note in enumerate(unique_notes)}

    X, y = [], []
    for song in notes:
        for i in range(len(song) - sequence_length):
            X.append([note_to_int[note] for note in song[i:i+sequence_length]])
            y.append(note_to_int[song[i+sequence_length]])

    n_vocab = len(unique_notes)
    X = np.reshape(X, (len(X), sequence_length, 1))
    X = X / n_vocab  # Normalización
    y = to_categorical(y, num_classes=n_vocab)

    return X, y, unique_notes, note_to_int


In [8]:
def build_model(sequence_length, n_vocab):
    model = Sequential([
        LSTM(512, input_shape=(sequence_length, 1), return_sequences=True),
        Dropout(0.3),
        LSTM(512, return_sequences=True),
        Dropout(0.3),
        LSTM(512),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(n_vocab, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model


In [9]:
def plot_history(history):
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.title('Model Performance')
    plt.legend()
    plt.show()


In [10]:
def generate_melody(model, start_sequence, int_to_note, melody_length=100):
    melody = start_sequence[:]
    for _ in range(melody_length):
        input_sequence = np.reshape(melody[-sequence_length:], (1, sequence_length, 1))
        prediction = model.predict(input_sequence, verbose=0)
        next_index = np.argmax(prediction)
        melody.append(next_index)
    return [int_to_note[i] for i in melody]


In [11]:
def save_melody_to_midi(melody, filename):
    output_notes = []
    for item in melody:
        if '.' in item or item.isdigit():
            notes_in_chord = item.split('.')
            chord_notes = [note.Note(n) for n in notes_in_chord]
            for n in chord_notes:
                n.storedInstrument = instrument.Piano()
            output_notes.append(chord.Chord(chord_notes))
        else:
            n = note.Note(item)
            n.storedInstrument = instrument.Piano()
            output_notes.append(n)
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)


In [ ]:
data_path = "midi"
notes = load_midi_files(data_path)

sequence_length = 50
X, y, unique_notes, note_to_int = prepare_sequences(notes, sequence_length)
int_to_note = {num: note for note, num in note_to_int.items()}

model = build_model(sequence_length, len(unique_notes))
history = model.fit(X, y, epochs=50, batch_size=64)




In [ ]:
plot_history(history)


In [ ]:

start_sequence = [note_to_int[note] for note in notes[0][:sequence_length]]
generated_melody = generate_melody(model, start_sequence, int_to_note)


In [ ]:
save_melody_to_midi(generated_melody, "generated_music.mid")